In [13]:
import psycopg2

url="localhost"
data_base="UNIDAD2_BDS"
user="postgres"
with open("C:/Users/Windows/Downloads/pwd_postgre.txt",'r') as f:
    pwd= f.read()

try:
    conn = psycopg2.connect(
        host=url,
        dbname=data_base,
        user=user,
        password=pwd,
        port='5432'
    )
    print("Connected to Postgres successfully!")
    
except Exception as e:
    print("Unable to connect to Postgre.")
    print(e)

Connected to Postgres successfully!


In [14]:
conn

<connection object at 0x000002CB76CDB260; dsn: 'user=postgres password=xxx dbname=UNIDAD2_BDS host=localhost port=5432', closed: 0>

In [15]:
import pandas as pd
ruta_archivos='./Tablas/'
product = pd.read_csv(ruta_archivos+'product.csv')
productreview = pd.read_csv(ruta_archivos+'productreview.csv')
productmodelproductdescriptionculture = pd.read_csv(ruta_archivos+'productmodelproductdescriptionculture.csv')
productdescription = pd.read_csv(ruta_archivos+'productdescription.csv')
salesorderdetail = pd.read_csv(ruta_archivos+'salesorderdetail.csv')
productcategory = pd.read_csv(ruta_archivos+'productcategory.csv')
productsubcategory = pd.read_csv(ruta_archivos+'productsubcategory.csv')
salesperson = pd.read_csv(ruta_archivos+'salesperson.csv')
salesorderheader = pd.read_csv(ruta_archivos+'salesorderheader.csv')
salesterritory = pd.read_csv(ruta_archivos+'salesterritory.csv')
countryregioncurrency = pd.read_csv(ruta_archivos+'countryregioncurrency.csv')
currencyrate = pd.read_csv(ruta_archivos+'currencyrate.csv')

In [16]:
product.dtypes

productid              int64
name                  object
productnumber         object
makeflag              object
finishedgoodsflag     object
                      ...   
sellstartdate         object
sellenddate           object
discontinueddate     float64
rowguid               object
modifieddate          object
Length: 25, dtype: object

El `type_map` lo puedes hacer tan sofisticado como quieras como esto:

```python
type_map = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'bool': 'BOOLEAN',
    'datetime64[ns]': 'TIMESTAMP',
    'timedelta64[ns]': 'INTERVAL',
    'complex128': 'VARCHAR(50)',
    'object': 'VARCHAR(50)',
    'category': 'VARCHAR(50)',
}
```

In [17]:
from psycopg2.extras import execute_values

def cargar_en_redshift(conn, table_name, dataframe):
    dtypes= dataframe.dtypes
    cols= list(dtypes.index )
    tipos= list(dtypes.values)
    type_map = {'int64': 'INT','float64': 'FLOAT','object': 'VARCHAR(50)'}
    sql_dtypes = [type_map[str(dtype)] for dtype in tipos]
    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f"{name} {data_type}" for name, data_type in zip(cols, sql_dtypes)]
    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_defs)}
        );
        """
    # Crear la tabla
    cur = conn.cursor()
    cur.execute(table_schema)
    # Generar los valores a insertar
    values = [tuple(x) for x in dataframe.to_numpy()]
    # Definir el INSERT
    insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES %s"
    # Execute the transaction to insert the data
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    cur.execute("COMMIT")
    print('Proceso terminado')

# Probando que funcione

In [19]:
cargar_en_redshift(conn=conn, table_name='product', dataframe=product)

Proceso terminado


In [20]:
cargar_en_redshift(conn=conn, table_name='currencyrate', dataframe=currencyrate)

Proceso terminado


In [21]:
cargar_en_redshift(conn=conn, table_name='productcategory', dataframe=productcategory)

Proceso terminado


In [22]:
conn.close()